### Setup a Conda Environment on Coiled

In [ ]:
import coiled

coiled.create_software_environment(
   name='mapshader-2',
   conda={
       'channels': ['conda-forge', 'defaults'],
       'dependencies': [
           'python=3.8',
           'mapshader=0.1.2',
           'dask=2022.04.2',
           'distributed=2022.4.2',
           'cloudpickle=2.0.0',
           'spatialpandas',
           'boto3',
       ],
    },
)

### Create Dask Cluster on Coiled 

In [ ]:
from coiled.v2 import Cluster
cluster = Cluster(name='mapshader-tiler',
                  n_workers=2,
                  worker_cpu=2,
                  worker_options={"nthreads": 1},
                  scheduler_memory="8 GiB",
                  software='mapshader-2')

from dask.distributed import Client
client = Client(cluster)
print('Dashboard:', client.dashboard_link)

### Clear cluster memory if necessary

In [ ]:
client.restart()

## Tile World Elevation

### Setup Mapshader Source

In [ ]:
from os import path

import geopandas as gpd
import mapshader
import spatialpandas

from mapshader.sources import RasterSource


def world_elevation_source():
    # file data path
    elevation_path = 's3://mapshader-testing/elevation.tif'
    # construct transforms
    squeeze_transform = dict(name='squeeze', args=dict(dim='band'))
    cast_transform = dict(name='cast', args=dict(dtype='float64'))
    orient_transform = dict(name='orient_array')
    flip_transform = dict(name='flip_coords', args=dict(dim='y'))
    reproject_transform = dict(name='reproject_raster', args=dict(epsg=3857))
    transforms = [
        squeeze_transform,
        cast_transform,
        orient_transform,
        flip_transform,
        reproject_transform,
    ]    
    # construct value obj
    source_obj = dict()
    source_obj['name'] = 'Elevation'
    source_obj['key'] = 'elevation'
    source_obj['text'] = 'Elevation'
    source_obj['description'] = 'Global Elevation Dataset'
    source_obj['geometry_type'] = 'raster'
    source_obj['shade_how'] = 'linear'
    source_obj['cmap'] = ['white', 'black']
    source_obj['span'] = 'min/max'
    source_obj['raster_padding'] = 0
    source_obj['raster_interpolate'] = 'linear'
    source_obj['xfield'] = 'geometry'
    source_obj['yfield'] = 'geometry'
    source_obj['filepath'] = elevation_path
    source_obj['transforms'] = transforms
    source_obj['service_types'] = ['tile', 'wms', 'image', 'geojson']
    source_obj['tiling'] = dict(min_zoom=1,
                                max_zoom=2,
                               )
    return source_obj


elevation_source = RasterSource.from_obj(world_elevation_source())
elevation_source.load()
elevation_source.data

## Visualize the source data

In [ ]:
elevation_source.data.plot.imshow()

### Create a Dask DataFrame of Tile to Process based on Map Source feature extents

In [ ]:
from mapshader.tile_utils import list_tiles

tiles_ddf = list_tiles(elevation_source, npartitions=2)
tiles_ddf.persist()
tiles_ddf

### Generate tile images and save to an S3 bucket

In [ ]:
from mapshader.tile_utils import save_tiles_to_outpath

save_tiles_to_outpath(elevation_source, tiles_ddf, outpath='s3://mapshader-tiling-test-999/elevation/')

### View tiles on OSM basemap

In [ ]:
from ipyleaflet import Map, TileLayer, basemaps, basemap_to_tiles

tiles_url = 'https://mapshader-tiling-test-999.s3.amazonaws.com/elevation/{z}/{x}/{y}.png'
tile_layer=TileLayer(url=tiles_url)

from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(48.204793, 350.121558),
    zoom=3
    )
m

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    zoom=2,
    scroll_wheel_zoom=True)

m.add_layer(tile_layer)
m
display(m)